# SciGRID Exploration

This notebook explores the SciGRID power system model using pandas.
It is part of the Smart Wires Products and Solutions Analytics
[open training course](https://github.com/smartwiresinc/training).

## 1. Import pandas

Load the Python Data Analysis library with the name `pd`

In [ ]:
import pandas as pd

## 2. Load and display data sets

### Vertices, or terminal nodes (substations, generators)

In [ ]:
terminals = pd.read_csv('vertices_de_power_160718.csv', quotechar="'")
terminals.head()

### Links, or branches connecting terminal nodes (transmission lines)

In [ ]:
branches = pd.read_csv('links_de_power_160718.csv', quotechar="'")
branches.head()

## 3. Try Jupyter

Use this area for the exercises.